In [1]:
import pandas as pd
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
tables = pd.read_html(url)

In [2]:
df = tables[0]
df.rename(columns={"Postcode":"Postal Code"}, inplace=True)
df.rename(columns={"Neighbourhood":"Neighborhood"}, inplace=True)

In [3]:
df.drop(df.loc[df["Borough"] == "Not assigned"].index, inplace=True)

In [4]:
df["Neighborhood"] = df.apply(
    lambda row: row["Borough"] if (row["Neighborhood"]=="Not assigned") else row["Neighborhood"],
    axis=1
)

In [6]:
df1 = df.groupby(["Postal Code", "Borough"],as_index=False,sort=False).agg(','.join)

In [10]:
df1.shape

(103, 3)

In [16]:
df1.head(10)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


In [8]:
geospatial = "http://cocl.us/Geospatial_data"
table2 = pd.read_csv(geospatial)

table2.head()
#merge
df_geospatial = pd.merge(df1,table2, how='inner', left_on = "Postal Code", right_on = "Postal Code")

In [17]:
df_geospatial.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937


In [12]:
df_geospatial.shape

(103, 5)

In [13]:
#!conda install -c conda-forge folium=0.8.3 --yes
import folium
print('Folium installed and imported!')

Folium installed and imported!


In [21]:
latitude = 43.651070
longitude = -79.347015
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=12)

# instantiate a feature group for the incidents in the dataframe
neighborhoods = folium.map.FeatureGroup()

# loop through the 103 neighborhoods to the incidents feature group
for lat, lng, in zip(df_geospatial.Latitude, df_geospatial.Longitude):
    neighborhoods.add_child(
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            color='cyan',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

# add incidents to map
toronto_map.add_child(neighborhoods)

In [20]:
toronto_map